## Generating two dimensional cluster states

In this notebook we generate a 1D cluster state on a ring

In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.sparse as sps

import cirq
from qutip import *

import qutrit_utils
import graph_state_gen_circuits as gsg

In [5]:
### system parameters
qus = [4,5,6,7,8,9,10]

waittimes = [0.125, 0.2, 0.125, 0.125, 0.075, 0.205, 0.275] # T_h, T_cz, T_pief, T_cnot, T_wait, T_swap1, T_swap2
coherence_times1 = [27, 22, 16, 12]
coherence_times2 = [22, 23, 4, 6]

nparams = [0.0, 0.02, 0.01, 0.0]

In [6]:
##-- initialize the simulators
simulator = cirq.Simulator()
dsim = cirq.DensityMatrixSimulator()

In [7]:
#gsg.leak_ring_cluster_state(3, 0.0, 0.0, 0.0)

In [8]:
#gsg.noisy_ring_cluster_state(3, waittimes, coherence_times1, coherence_times2, nparams)

## Generating the states

In [9]:
%%time
### the ideal states
noiseless_states = []

for qu in qus:
    #- run the circuit
    ideal_run = simulator.simulate(gsg.leak_ring_cluster_state(qu,0,0,0))
    
    #- ideal state in csr form
    ideal_rho = sps.csr_matrix(np.outer(np.conj(ideal_run.final_state_vector), ideal_run.final_state_vector))
    
    #- ideal state as qutip object
    irho = Qobj(ideal_rho, dims = [[3,3] + [2]*qu, [3,3] + [2]*qu])
    
    #- reduced state of the qubit register (trace out the storages)
    irho_qubits = ptrace(irho, list(range(2, qu+2)))
    
    noiseless_states.append(irho_qubits)
    
    print("Computation for %d qubits, done -->>" % qu)
#

Computation for 4 qubits, done -->>
Computation for 5 qubits, done -->>
Computation for 6 qubits, done -->>
Computation for 7 qubits, done -->>
Computation for 8 qubits, done -->>
Computation for 9 qubits, done -->>
Computation for 10 qubits, done -->>
CPU times: user 2.28 s, sys: 233 ms, total: 2.51 s
Wall time: 2.52 s


In [10]:
%%time
### the decoherence limited states
deco_states = []

for qu in qus:
    #- run the circuit
    deco_run = dsim.simulate(gsg.noisy_ring_cluster_state(qu, waittimes, coherence_times1, coherence_times2, [0,0,0,0]))
    
    #- deco state in csr form
    deco_rho = sps.csr_matrix(deco_run.final_density_matrix)
    
    #- ideal state as qutip object
    drho = Qobj(deco_rho, dims = [[3,3] + [2]*qu, [3,3] + [2]*qu])
    
    #- reduced state of the qubit register (trace out the storages)
    drho_qubits = ptrace(drho, list(range(2, qu+2)))
    
    deco_states.append(drho_qubits)
    print("Computation for %d qubits, done -->>" % qu)
#

Computation for 4 qubits, done -->>
Computation for 5 qubits, done -->>
Computation for 6 qubits, done -->>
Computation for 7 qubits, done -->>
Computation for 8 qubits, done -->>
Computation for 9 qubits, done -->>
Computation for 10 qubits, done -->>
CPU times: user 49.6 s, sys: 6.13 s, total: 55.7 s
Wall time: 51.6 s


In [11]:
%%time
### the noisy states
noisy_states = []

for qu in qus:
    #- run the circuit
    noisy_run = dsim.simulate(gsg.noisy_ring_cluster_state(qu, waittimes, coherence_times1, coherence_times2, nparams))
    
    #- deco state in csr form
    noisy_rho = sps.csr_matrix(noisy_run.final_density_matrix)
    
    #- ideal state as qutip object
    nrho = Qobj(noisy_rho, dims = [[3,3] + [2]*qu, [3,3] + [2]*qu])
    
    #- reduced state of the qubit register (trace out the storages)
    nrho_qubits = ptrace(nrho, list(range(2, qu+2)))
    
    noisy_states.append(nrho_qubits)
    
    print("Computation for %d qubits, done -->>" % qu)
#

Computation for 4 qubits, done -->>
Computation for 5 qubits, done -->>
Computation for 6 qubits, done -->>
Computation for 7 qubits, done -->>
Computation for 8 qubits, done -->>
Computation for 9 qubits, done -->>
Computation for 10 qubits, done -->>
CPU times: user 1min 1s, sys: 20 s, total: 1min 21s
Wall time: 52.5 s


### Fidelities

In [ ]:
fides_deco = []
fides_noisy = []

for qq in range(len(qus)):
    fdeco = fidelity(deco_states[qq], noiseless_states[qq])**2
    ndeco = fidelity(noisy_states[qq], noiseless_states[qq])**2
    
    fides_deco.append(fdeco)
    fides_noisy.append(ndeco)
    
    print(qus[qq], fdeco, ndeco)
#

4 0.9226264531204315 0.838034275532221
5 0.893948466481114 0.8099306872817692
6 0.8808801168573643 0.7847719466837949
7 0.8535166156915052 0.7450390197863849
8 0.841042134046783 0.7186867108995506
9 0.814917005378726 0.6825852666153295


In [ ]:
plt.plot(qus, fides_deco, "o-", label = "simulation - decoherence limited")
plt.plot(qus, fides_noisy, "o-", label = "simulation - leakage")

plt.legend(loc = "upper right", fontsize = 12, frameon = False)
plt.xlabel("Number of qubits", fontsize = 16)
plt.ylabel("Fidelity", fontsize = 16)
plt.tick_params(axis = "both", labelsize = 13)
plt.show()

## Exporting the states

In [ ]:
### path
#patho = "/Users/munm2002/Documents/projects/graph_states_for_ent_witness/data_states/cluster_state_ring/"

In [ ]:
### save the states
# for ii in range(len(qus)):

#     qsave(noisy_states[ii], patho+"cluster_state_ring_%dqubits.qu" % qus[ii])

#     print("Size %d, completed -->>" % (qus[ii]))
# #

In [ ]:
# ### checking the stabilizers
# #- signle site operators
# sx = sigmax()
# sz = sigmaz()
# ide = identity(dims = [2])

# #- stabilizer generators (for three photons ring)
# S1 = tensor([sx, sz, sz])
# S2 = tensor([sz, sx, sz])
# S3 = tensor([sz, sz, sx])

# #- compute expectations
# e1 = expect(S1, noiseless_states[0])
# e2 = expect(S2, noiseless_states[0])
# e3 = expect(S3, noiseless_states[0])

# print(e1)
# print(e2)
# print(e3)